# Deep Neural Networks

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
# Python 2 and 3 Compatibility Library
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = "DeepLearning/data/notMNIST.pickle"

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  
  train_data = save['train_dataset']
  train_target = save['train_labels']
  
  valid_data = save['valid_dataset']
  valid_target = save['valid_labels']
  
  test_data = save['test_dataset']
  test_target = save['test_labels']
  
  del save  

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [15]:
def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
    if padding == 'SAME':
        padding = -1.00
    elif padding == 'VALID':
        padding = 0.00
    else:
        return None
    # Dopo la prima convoluzione
    output_1 = (((input_size - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # Dopo il primo pooling
    # Uso la stessa formula poiche' il pooling ha lo stride pari a 2, 
    # percio' riduce l'input come la convoluzione
    output_2 = (((output_1 - pool_filter_size - 2*padding) / pool_stride) + 1.00)    
    # Dopo la seconda convoluzione
    output_3 = (((output_2 - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # Dopo il secondo pooling
    output_4 = (((output_3 - pool_filter_size - 2*padding) / pool_stride) + 1.00)  
    return int(output_4)


### IPERPARAMETRI

In [13]:
#
# INPUT
#
# La dimensione delle immagini 28x28
image_size = 28
# la dimensione della colonna target 10x1
num_labels = 10
# La profondita' dei dati in input
num_channels = 1 
# 1 = scala di grigi

#
# DATASET 
#
train_data, train_target = reformat(train_data, train_target)
valid_data, valid_target = reformat(valid_data, valid_target)
test_data,   test_target = reformat(test_data,   test_target)

#
# FILTERS
#
# Dimensione del minibatch
batch_size = 16
# Dimensione della finestra per la convoluzione 5x5
patch_size = 5
# Numero di features (o profondita') del filtro
depth = 16
# Nodi dell'hidden layer
num_hidden = 64
# Numero di celle che il filtro scorre ad ogni iterazione
stride = 1
# Dimensione della finestra per il pooling 2x2
pool_size = 2 
# Numero di celle che il pooling scorre ad ogni iterazione
pool_stride = 2
# Probabilita' del Dropout
keep_prob = 0.5

### ARCHITETTURA TENSORFLOW

In [16]:
graph = tf.Graph()

with graph.as_default():
    
    # Per i minibatch utilizzo **placeholder**
    tf_train_data = tf.placeholder(tf.float32,
                              shape=(
                                  batch_size, image_size, num_channels))
    tf_train_target = tf.placeholder(tf.float32,
                              shape=(
                                  batch_size,             num_channels))
    
    # Senza minibatch
    tf_valid_data = tf.constant(valid_data)
    tf_test_data  = tf.constant(test_data)
    
    # Layer1 CNN
    # Dimens   = l_finestra, h_finestra, prof_input, prof_output
    dim_layer1 = [patch_size, patch_size, num_channels, depth]
    # Pesi RANDOM + TRUNCATE valori troppo bassi o alti
    tf_layer1_pesi = tf.Variable(tf.truncated_normal(dim_layer1, stddev=0.1))
    # Bias RANDOM (0)
    tf_layer1_bias = tf.Variable(tf.zeros([depth]))
    
    # Layer2 CNN
    # Dimens   = l_finestra, h_finestra, prof_input, prof_output
    dim_layer2 = [patch_size, patch_size, depth, depth]
    # Pesi RANDOM + TRUNCATE 
    tf_layer2_pesi = tf.Variable(tf.truncated_normal(dim_layer2, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer2_bias = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Layer3 FULL
    # Ricalcolo la dimensione del layer in base alla nuova configurazione 
    ##final_image_size = output_size_pool(image_size, patch_size, pool_size, 
    ##                                    'VALID', stride, pool_stride)
    # Dimens   = l_input(dopo2CNN), h_input(dopo2CNN), prof_input, prof_output
    dim_layer3 = [image_size/4, image_size/4, depth, num_hidden]
    # Pesi RANDOM + TRUNCATE 
    tf_layer3_pesi = tf.Variable(tf.truncated_normal(dim_layer3, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer3_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer4 FULL
    # Dimens   = lunghezza, altezza
    dim_layer4 = [num_hidden, num_hidden]
    # Pesi RANDOM + TRUNCATE 
    tf_layer4_pesi = tf.Variable(tf.truncated_normal(dim_layer4, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer4_bias = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer5 FULL
    # Dimens   = lunghezza, altezza
    dim_layer5 = [num_hidden, num_labels]
    # Pesi RANDOM + TRUNCATE 
    tf_layer5_pesi = tf.Variable(tf.truncated_normal(dim_layer5, stddev=0.1))
    # Bias RANDOM (1)
    tf_layer5_bias = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    #
    # MODELLO
    # 
    def model(data):
        # Convolutional LAYER1
        conv_1 = tf.nn.conv2d(data, tf_layer1_pesi, 
                              strides=[1, stride, stride, 1], 
                              padding='VALID')
        # RELU LAYER1
        hidden_1 = tf.nn.relu(conv_1 + tf_layer1_bias)
        # Average POOLING LAYER1
        pool_1 = tf.nn.avg_pool(hidden_1, 
                                [1, pool_size, pool_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Convolutional LAYER2
        conv_2 = tf.nn.conv2d(pool_1, tf_layer2_pesi, 
                              strides=[1, stride, stride, 1], 
                              padding='VALID')
        # RELU LAYER2
        hidden_2 = tf.nn.relu(conv_2 + tf_layer2_bias)
        # Average POOLING LAYER2
        pool_2 = tf.nn.avg_pool(hidden_2, 
                                [1, pool_size, pool_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Full Connected LAYER3
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, 
                            [shape[0], shape[1] * shape[2] * shape[3]])
        # RELU LAYER3
        hidden_3 = tf.nn.relu(tf.matmul(reshape, 
                                      tf_layer3_pesi) + tf_layer3_bias)
        # DROPOUT LAYER3
        hidden_drop_3 = tf.nn.dropout(hidden_3, keep_prob)
        
        # Full Connected RELU LAYER4
        hidden_4 = tf.nn.relu(tf.matmul(hidden_drop_3, 
                                        tf_layer4_pesi) + tf_layer4_bias)
        # DROPOUT LAYER4
        hidden_drop_4 = tf.nn.dropout(hidden_4, keep_prob)
        
        return tf.matmul(hidden_drop_4, tf_layer5_pesi) + tf_layer5_bias
    
    # ESECUZIONE
    logits = model(train_data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                                    labels=tf_train_target, logits=logits))  
    # REGULARIZZATION
    # N.B. La regolarizzazione si applica solitamente alle reti full-connected.
    regularizers = tf.nn.l2_loss(tf_layer4_pesi) + tf.nn.l2_loss(tf_layer5_pesi)
    loss = tf.reduce_mean(loss + 0.001 * regularizers)
    
    # DECAY
    global_step = tf.Variable(0)  
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, 
                                               global_step, 
                                               100000, 
                                               0.96, 
                                               staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # ADDESTRAMENTO
    train_prediction = tf.nn.softmax(logits)
    # VALIDAZIONE e TEST
    valid_prediction = tf.nn.softmax(model(valid_data))
    test_prediction = tf.nn.softmax(model(test_data))

ValueError: Shape must be rank 2 but is rank 4 for 'MatMul' (op: 'MatMul') with input shapes: [200000,256], [4,4,16,64].